<a href="https://colab.research.google.com/github/SaraMohamed-121/CV-ST-24/blob/main/notebook5888ca8c77.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'appl-ai-cv-st-24-assignment-1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F83802%2F9339243%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T190356Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D238bf3a30f4d0e1e00a8287f698a021d2303ee79426343378061af946464c9a8f3753f57b3ac5d315aaa7be5ef07b97bffd584872d8a64084ed233eaccb3a80c417b0c006c71d3b716f3da91e0067c98a751c39856f69598efc49f3fbab1564279835647a2820abb4d9f5d747b1504d353fbae0d9f986ab00ccea4bf419f98425182df2931724cb142685fd0b47c3c8047a07bcc8a949e6991e3bfc6f7a1175e74815213fd93e148420d9762b9abc872a698ac44b35625ae9cea28712f21ddcbe0b3cefce14f7cab78d4e4c27637b28b2ce7cce88a79e0dd7080e0affaa6d66ea2e1fcdab83162787a01f4bc68470ea9a3cdc96baa8f5512dce00b8e069f8d67'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import joblib

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'

# Load data
train_path = '/kaggle/input/appl-ai-cv-st-24-assignment-1/train.csv'
valid_path = '/kaggle/input/appl-ai-cv-st-24-assignment-1/test.csv'
traindata = pd.read_csv(train_path)
valldata = pd.read_csv(valid_path)
print(f'train data: {traindata.shape}\nval data: {valldata.shape}')
traindata.info()

In [ ]:
#dataset contains 103904 examples and 25 columns, 24 features
from sklearn.model_selection import train_test_split
traindata, testdata = train_test_split(traindata, test_size=0.3, random_state=42)
traindata.head(5)

In [ ]:
for col in traindata.columns:
    if traindata[col].dtype == 'object':
        #print(col)
        traindata[col].value_counts().plot(kind='bar', figsize=(5,5))
        plt.title(col)
        plt.xlabel(col)
        plt.ylabel('Count')
        plt.show()
print(traindata['satisfaction'])
# traindata['Gender'].value_counts().plot(kind='bar', figsize=(5,5))
# traindata['Customer Type'].value_counts().plot(kind='bar', figsize=(5,5))
# traindata['Type of Travel'].value_counts().plot(kind='bar', figsize=(5,5))
# traindata['Class'].value_counts().plot(kind='bar', figsize=(5,5))
# traindata['satisfaction'].value_counts().plot(kind='bar', figsize=(5,5))

In [ ]:
satisfaction_mapping = {'satisfied': 1, 'neutral or dissatisfied': 0}
traindata['satisfaction'] = traindata['satisfaction'].map(satisfaction_mapping)
testdata['satisfaction'] = testdata['satisfaction'].map(satisfaction_mapping)
# prediction train data
if not traindata['satisfaction'].empty:
    traindata['satisfaction'].value_counts().plot(kind='bar', figsize=(5, 5), color='red')
    plt.xlabel('Satisfaction')
    plt.ylabel('Count')
    plt.title('Distribution of Satisfaction')
    plt.show()
else:
    print("No data to plot.")

In [ ]:
sns.kdeplot(data=traindata, x='satisfaction', color='blue')
plt.xlabel('Satisfaction')
plt.title('Satisfaction')
plt.show()
skewness = traindata['satisfaction'].skew()
print(skewness)

sns.kdeplot(data=traindata, x='id', color='blue')
plt.xlabel('id')
plt.title('id')
plt.show()

In [ ]:
def get_feats_and_labels(data, label):
    data_feats = data.drop(label, axis=1)
    data_label = data[label]
    return data_feats, data_label

train_feats, train_label = get_feats_and_labels(traindata, 'satisfaction')
train_feats
train_label

In [ ]:
traindata = traindata.drop(['Unnamed: 0', 'Gender' ,'id' , 'Age'], axis=1)
traindata.head(5)

In [ ]:
scaler = StandardScaler()
encoder = OrdinalEncoder()


# The column transformer requires lists of features
num_feats = ['Age', 'Flight Distance', 'Inflight wifi service',
             'Departure/Arrival time convenient', 'Ease of Online booking',
             'Gate location', 'Food and drink', 'Online boarding',
             'Seat comfort', 'Inflight entertainment', 'On-board service',
             'Leg room service', 'Baggage handling', 'Checkin service',
             'Inflight service', 'Cleanliness', 'Departure Delay in Minutes',
             'Arrival Delay in Minutes']

cat_feats = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# define the pipeline to scale the numeric features and handle categorical features
final_pipe = ColumnTransformer([
   ('num',scaler , num_feats),
   ('cat', encoder , cat_feats)

])
training_data_prepared = final_pipe.fit_transform(train_feats)
print(training_data_prepared.shape)
type(training_data_prepared)


In [ ]:
test_feats, test_label = get_feats_and_labels(testdata, 'satisfaction')
test_feats
test_label

In [ ]:
test_data_prepared = final_pipe.transform(test_feats)
test_data_prepared

In [ ]:
# train_label = train_label.to_numpy()
# test_label = test_label.to_numpy()
# Check if train_label is already a numpy array
if isinstance(train_label, np.ndarray):
    # No need to convert, just assign
    train_label = train_label
else:
    # Convert to numpy array if it's a DataFrame or Series
    train_label = train_label.to_numpy()

# Repeat for test_label
if isinstance(test_label, np.ndarray):
    test_label = test_label
else:
    test_label = test_label.to_numpy()


train_label
test_label

In [ ]:
input_shape = training_data_prepared.shape[1:]
input_shape

In [ ]:
model = keras.models.Sequential([

            # The first layers must specify the input shape always
            keras.layers.Dense(24, activation='relu', input_shape=input_shape),
            keras.layers.Dense(48, activation='relu'),
            keras.layers.Dense(96, activation='relu'),
            keras.layers.Dense(192, activation='relu'),
            # The last layer usually doesn't have activation function in regression
            keras.layers.Dense(1)

])

# Now we compile the model
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
import numpy as np

# Convert labels to float64
train_label = train_label.astype('float64')
test_label = test_label.astype('float64')

# Print data types to confirm
print("training_data_prepared dtype:", training_data_prepared.dtype)
print("train_label dtype:", train_label.dtype)
print("test_data_prepared dtype:", test_data_prepared.dtype)
print("test_label dtype:", test_label.dtype)

# Handle NaN values in numpy arrays
if np.isnan(training_data_prepared).any():
    print("Data contains NaN values in training_data_prepared. Cleaning the data...")
    training_data_prepared = np.where(np.isnan(training_data_prepared),
                                      np.nanmean(training_data_prepared, axis=0),
                                      training_data_prepared)

if np.isnan(test_data_prepared).any():
    print("Data contains NaN values in test_data_prepared. Cleaning the data...")
    test_data_prepared = np.where(np.isnan(test_data_prepared),
                                  np.nanmean(test_data_prepared, axis=0),
                                  test_data_prepared)

# Verify that the NaN values are handled
if np.isnan(training_data_prepared).any():
    print("NaN values still exist in training data.")
else:
    print("Training data is clean.")

if np.isnan(test_data_prepared).any():
    print("NaN values still exist in test data.")
else:
    print("Test data is clean.")

In [ ]:
# Fit the model again
history = model.fit(training_data_prepared, train_label,
                    validation_data=(test_data_prepared, test_label),
                    epochs=100)

In [ ]:
loss_df = pd.DataFrame(history.history)
loss_df.plot(figsize=(10,5))

In [ ]:
model.evaluate(test_data_prepared, test_label)

In [ ]:
sns.kdeplot(data=traindata, x='satisfaction', color='red')

In [ ]:
model_2 = keras.models.Sequential([

            # The first layers must specify the input shape always
            keras.layers.Dense(12, activation='relu', input_shape=input_shape),
            keras.layers.Dense(24, activation='relu'),
            keras.layers.Dense(6, activation='relu'),
            keras.layers.Dense(3, activation='relu'),
            # The last layer usually doesn't have activation function in regression
            keras.layers.Dense(1, activation='sigmoid')

])

# Now we compile the model
model_2.compile(loss='log_cosh', optimizer='adam')
history = model_2.fit(training_data_prepared, train_label,
                    validation_data = (test_data_prepared, test_label), epochs=100)

In [ ]:
loss_df = pd.DataFrame(history.history)
loss_df.plot(figsize=(10,5))

In [ ]:
model.evaluate(test_data_prepared, test_label)

In [ ]:
model.save('f_model.h5')
print("done")

In [ ]:
loaded_model = keras.models.load_model('f_model.h5')
loss_acc_metrics_df = pd.DataFrame(history.history)
loss_acc_metrics_df.plot(figsize=(10,5))

In [ ]:
# Evaluate the model
model_eval = model_2.evaluate(test_data_prepared, test_label)

# Check if the evaluation returned a single value or multiple values
if isinstance(model_eval, float):
    print(f"Test loss: {model_eval}")
else:
    # If the evaluation returns a list/tuple, assuming [loss, accuracy]
    print(f"Test loss: {model_eval[0]}\nTest accuracy: {model_eval[1]}")

In [ ]:
# Define a function to extract features (dropping the target column if present)
def get_feats(data):
    return data.drop(['Unnamed: 0', 'satisfaction'], axis=1, errors='ignore')

# Define the numeric and categorical features (ensure these match your data)
num_feats = ['Age', 'Flight Distance', 'Inflight wifi service',
             'Departure/Arrival time convenient', 'Ease of Online booking',
             'Gate location', 'Food and drink', 'Online boarding',
             'Seat comfort', 'Inflight entertainment', 'On-board service',
             'Leg room service', 'Baggage handling', 'Checkin service',
             'Inflight service', 'Cleanliness', 'Departure Delay in Minutes',
             'Arrival Delay in Minutes']

cat_feats = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# Define the pipeline to handle missing values, scale numeric features, and encode categorical features
final_pipe = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values in numeric features
        ('scaler', StandardScaler())
    ]), num_feats),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values in categorical features
        ('encoder', OneHotEncoder())
    ]), cat_feats)
])

train_target = traindata['satisfaction']

# Fit the pipeline on the training data features
final_pipe.fit(train_feats)

# Transform training features using the fitted pipeline
train_feats_prepared = final_pipe.transform(train_feats)

# Train your model (example using RandomForestClassifier)
model = RandomForestClassifier()
model.fit(train_feats_prepared, train_target)

# Save the pipeline and model
joblib.dump(final_pipe, 'final_pipe.pkl')
joblib.dump(model, 'your_model.pkl')


# Get the features from the validation data
val_feats = get_feats(valldata)

# Prepare the validation data using the already fitted pipeline
val_data_prepared = final_pipe.transform(val_feats)

# Load your previously saved model
loaded_model = joblib.load('your_model.pkl')

# Make predictions on validation data
predictions = loaded_model.predict(val_data_prepared)
print(predictions[:15])



In [ ]:
# If predictions is a numpy array of strings, convert it using the mapping
predictions_mapped = np.array([satisfaction_mapping.get(pred, pred) for pred in predictions])

#ids = np.arange(0, 25976)
ids = valldata['id']
# Create a DataFrame with 'id' and 'y' (predictions)
output_df = pd.DataFrame({
    'id': ids,
    'y': predictions_mapped
})
print(output_df)
print(predictions_mapped.shape)
# Save the DataFrame to a CSV file
output_df.to_csv('predictions.csv', index=False)
print("Predictions saved to 'predictions.csv'")